- https://colab.research.google.com/drive/1F_RNcHzTfFuQf-LeKvSlud6x7jXYkG31#scrollTo=goRmGIRI5cfC

# setup

In [1]:
HOME = "/data/git/product-category"
# http://localhost:8080/notebooks/git/product-category/notebooks/prep_20210304A1.ipynb
prfx_prp = 'prep_20210304A1'


p_out = f'{HOME}/data/transformer_20210305C1'
!mkdir -p {p_out}

# eda 

In [2]:
import pandas as pd
import numpy as np
from collections import Counter

In [3]:
%%time
# df = pd.read_csv(f'../data/data__{prfx_prp}.csv')
df = pd.read_csv(f'{HOME}/data/data_sample__{prfx_prp}.csv')

print(df.shape)
df.sample(3)

(10000, 9)
CPU times: user 141 ms, sys: 20.2 ms, total: 161 ms
Wall time: 159 ms


,category,description,title,brand,feature,asin,domain,txt,is_validation
8131,Sports & Outdoors|Sports & Fitness|Airsoft & P...,Collectible Military Scope Mount. Details: Lig...,Sun Optics USA Collectible Military Base Pre D...,Sun Optics USA,Collectible Military Scope Mount\nDetails: Lig...,B004LYJ6CY,Sports_and_Outdoor,Sun Optics USA Collectible Military Base Pre D...,0
6604,Kindle Store|Kindle eBooks|Children's eBooks,NaN,Andersen&#39;s Stories: The Little Mermaid &am...,Hans Christian Andersen,NaN,B006WO6K0W,Kindle_Store,Andersen&#39;s Stories: The Little Mermaid &am...,0
8524,Home & Kitchen|Kitchen & Dining|Kitchen & Tabl...,THE SWEDISH ECO-FRIENDLY DISHCLOTH: The dry sp...,"Swedish Dishcloth, Red &amp; Dark Turquoise - ...",Trendy Tripper,Cotton\nImported\nContemporary Swedish Dish Cl...,B00PJBOCPO,Home_and_Kitche,"Swedish Dishcloth, Red &amp; Dark Turquoise - ...",0


In [4]:
MIN_CNT = 50
dmn2cnt = Counter(df.domain.value_counts().to_dict())
i2dmn = sorted(dmn2cnt.keys())
dmn2i = {v:k for k,v in enumerate(i2dmn)}
cat2cnt = Counter((j for i in df.category.apply(lambda x: x.split('|')) for j in i))
i2cat = sorted(k for k,v in cat2cnt.items() if v>50)
cat2i = {v:k for k,v in enumerate(i2cat)}

print("len(i2dmn), len(i2cat)", len(i2dmn), len(i2cat))
print("|".join(i2dmn))
print()
print("|".join(i2cat))

len(i2dmn), len(i2cat) 23 111
Appliance|Arts_Crafts_and_Sewi|Automotive|Book|CDs_and_Vinyl|Cell_Phones_and_Accessorie|Clothing_Shoes_and_Jewelry|Electronic|Grocery_and_Gourmet_Food|Home_and_Kitche|Industrial_and_Scientific|Kindle_Store|Magazine_Subscripti|Movies_and_TV|Musical_Instrument|Office_Product|Patio_Lawn_and_Garde|Pet_Supplie|Software|Sports_and_Outdoor|Tools_and_Home_Improvement|Toys_and_Game|Video_Game

100% Cotton|100% Leather|100% Polyester|Accessories|Accessories & Supplies|Action Figures & Statues|Americas|Arts &amp; Photography|Arts, Crafts & Sewing|Athletic|Audio & Video Accessories|Automotive|Baby|Baby Girls|Bags, Cases & Sleeves|Basic Cases|Bedding|Biographies &amp; Memoirs|Body & Trim|Books|Boots|Boys|Bracelets|Bumper Stickers, Decals & Magnets|CDs & Vinyl|Camera & Photo|Cases, Holsters & Sleeves|Casual|Cell Phones & Accessories|Children's Books|Christian Books &amp; Bibles|Clothing|Clothing, Shoes & Jewelry|Computer Accessories & Peripherals|Computers & Accessories

# dataset

In [5]:
from argparse import ArgumentParser
import pytorch_lightning as pl
from transformers.optimization import AdamW

from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer
import torch

def mk_tensors(txt, tokenizer, max_seq_length):
    tok_res = tokenizer(
        txt, truncation=True, padding='max_length', max_length=max_seq_length
    )
    input_ids = tok_res["input_ids"]
    attention_mask = tok_res["attention_mask"]
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    return input_ids, attention_mask

def mk_ds(txt, tokenizer, max_seq_length, ys):
    input_ids, attention_mask = mk_tensors(txt, tokenizer, max_seq_length)
    return TensorDataset(input_ids, 
                         attention_mask, 
                         torch.tensor(ys)) 

class PCDataModule(pl.LightningDataModule):
    def __init__(self, 
                 model_name_or_path, 
                 max_seq_length, 
                 min_products_for_category,
                 train_batch_size,
                 val_batch_size,
                 data_file_path=None,
                 dataframe=None):
        super().__init__()
        self.data_file_path = data_file_path
        self.dataframe = dataframe
        self.min_products_for_category = min_products_for_category
        self.model_name_or_path = model_name_or_path
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.num_classes = None
      
    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoTokenizer.from_pretrained(self.model_name_or_path)

    def setup(self, stage=None):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path)
        if self.dataframe is None:
            self.dataframe = pd.read_csv(self.data_file_path)
        cats = self.dataframe.category.apply(lambda x: x.split('|'))
        cat2cnt = Counter((j for i in cats for j in i))
        i2cat = sorted(k for k,v in cat2cnt.items() if v>self.min_products_for_category)
        cat2i = {v:k for k,v in enumerate(i2cat)}
        self.num_classes = len(i2cat)
        self.i2cat, self.cat2i = i2cat, cat2i
        
        ys = np.zeros((len(self.dataframe), len(i2cat)))
        for i,cats in enumerate(self.dataframe.category):
            idx_pos = [cat2i[cat] for cat in cats.split('|') if cat in cat2i]
            ys[i,idx_pos] = 1
        
        msk_val = self.dataframe.is_validation==1
        self.df_trn = self.dataframe[~msk_val]
        self.df_val = self.dataframe[msk_val]
        idx_trn = np.where(~msk_val)[0]
        idx_val = np.where(msk_val)[0]
        self.ys_trn, self.ys_val = ys[idx_trn], ys[idx_val]
        
        txt = self.dataframe.txt.values
        self.train_dataset = mk_ds(list(self.df_trn.txt), self.tokenizer, self.max_seq_length, self.ys_trn)
        self.eval_dataset  = mk_ds(list(self.df_val.txt), self.tokenizer, self.max_seq_length, self.ys_val)
        
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
        )

    def val_dataloader(self):
        return DataLoader(
            self.eval_dataset,
            batch_size=self.val_batch_size,
        )

# model

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel


def getaccu(logits, ys):
    return ((torch.sigmoid(logits)>0.5).int()==ys).float().mean()

class PCModel(pl.LightningModule):
    def __init__(self, model_name_or_path, num_classes, learning_rate, adam_beta1, adam_beta2, adam_epsilon):
        super().__init__()
        self.save_hyperparameters()
        self.model_name_or_path = model_name_or_path
        self.bert = AutoModel.from_pretrained(self.model_name_or_path)
        self.num_classes = num_classes
        self.W = nn.Linear(self.bert.config.hidden_size, self.num_classes)

    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoModel.from_pretrained(self.model_name_or_path)

    def forward(self, input_ids, attention_mask):
        h = self.bert(input_ids, attention_mask)['last_hidden_state']
        h_cls = h[:, 0]
        return self.W(h_cls)
    
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        accu = getaccu(logits, ys)
        self.log('train_loss', loss, on_epoch=True)
        self.log('train_accu', accu, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        accu = getaccu(logits, ys)
        self.log('valid_loss', loss, on_step=False, sync_dist=True)
        self.log('valid_accu', accu, on_step=False, sync_dist=True)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(),
                          self.hparams.learning_rate,
                          betas=(self.hparams.adam_beta1,
                                 self.hparams.adam_beta2),
                          eps=self.hparams.adam_epsilon,)
        return optimizer    
    
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', type=float, default=5e-5)
        parser.add_argument('--adam_beta1', type=float, default=0.9)
        parser.add_argument('--adam_beta2', type=float, default=0.999)
        parser.add_argument('--adam_epsilon', type=float, default=1e-8)
        return parser

# train

In [10]:
parser = ArgumentParser()

parser.add_argument('--model_name_or_path', type=str,
                    default="distilbert-base-cased")
parser.add_argument('--max_seq_length', type=int, default=128)
parser.add_argument('--min_products_for_category', type=int, default=100)
parser.add_argument('--train_batch_size', type=int, default=16)
parser.add_argument('--val_batch_size', type=int, default=8)

parser = pl.Trainer.add_argparse_args(parser)
parser = PCModel.add_model_specific_args(parser)

args = parser.parse_args([
    '--default_root_dir', p_out,
])


data_module = PCDataModule(
    model_name_or_path=args.model_name_or_path,
#     data_file_path=f'{HOME}/data/data_sample__{prfx_prp}.csv',
    data_file_path=f'{HOME}/data/data__{prfx_prp}.csv',
    min_products_for_category=args.min_products_for_category,
    max_seq_length=args.max_seq_length,
    train_batch_size=args.train_batch_size,
    val_batch_size=args.val_batch_size,
)

data_module.prepare_data()

In [ ]:
%%time
data_module.setup()

/data/anaconda3/envs/product-category/lib/python3.7/site-packages/IPython/core/magics/execution.py:1313: DtypeWarning: Columns (1,2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  out = eval(code, glob, local_ns)


In [ ]:
pcmodel = PCModel(
    model_name_or_path=args.model_name_or_path,
    num_classes= data_module.num_classes,
    learning_rate=args.learning_rate,
    adam_beta1=args.adam_beta1,
    adam_beta2=args.adam_beta2,
    adam_epsilon=args.adam_epsilon,
)
pcmodel.prepare_data()

In [ ]:
dl = data_module.val_dataloader()
n=0
for dat in dl:
    input_ids, attention_mask, ys = dat

    logits = pcmodel(input_ids, attention_mask)

    accu = ((torch.sigmoid(logits)>0.5).int()==ys).float().mean()
    print(accu)
    n+=1
    if n>5: break

## run train

In [ ]:
pl.seed_everything(1234)
# trainer = pl.Trainer.from_argparse_args(args)
# trainer = pl.Trainer.from_argparse_args(args, fast_dev_run=True)
# trainer = pl.Trainer.from_argparse_args(args, limit_train_batches=10, limit_val_batches=5, gpus=1)
# trainer = pl.Trainer.from_argparse_args(args, gpus=1)
trainer = pl.Trainer.from_argparse_args(args, 
#                                         limit_train_batches=10, limit_val_batches=5, 
                                        max_epochs=1,
#                                         fast_dev_run=True, 
                                        log_gpu_memory=True, gpus=1)


In [ ]:
trainer.fit(pcmodel, data_module)

## tensorboard

In [ ]:
!ls {p_out}/lightning_logs/

# eval

In [ ]:
dl = data_module.val_dataloader()
n=0
for dat in dl:
    input_ids, attention_mask, ys = dat

    logits = pcmodel(input_ids, attention_mask)

    accu = ((torch.sigmoid(logits)>0.5).int()==ys).float().mean()
    print(accu)
    n+=1
    if n>5: break

# fin

In [ ]:
!nvidia-smi